In [0]:
%pip install --upgrade --force-reinstall -r https://aka.ms/automl_linux_requirements.txt

In [0]:
import azureml.core

print("SDK Version:", azureml.core.VERSION)

In [0]:
# define widgets - NEED TO DEFINE IT ONCE
# dynamic variables (pass it from ADF)
# first time runtime parameter
# dbutils.widgets.dropdown("environment", "dev", ["dev","uat","prod"])
# dbutils.widgets.dropdown("new_training", "False", ["True","False"])
# dbutils.widgets.dropdown("system_name", "bimodelapi", ["bimodelapi"])
# dbutils.widgets.text("system_name", "","")
# dbutils.widgets.remove("pbiapi")
environment = dbutils.widgets.get("environment")
new_training = dbutils.widgets.get("new_training")
system_name = dbutils.widgets.get("system_name")
if environment in {"prod"}:
    aml_compute_cluster_name = "cc-bi-ml-prod01"
else:
    aml_compute_cluster_name = "cc-bi-ml-devqa01"

print(environment, system_name, new_training, aml_compute_cluster_name)


In [0]:
%run /bi_config/lib_helper

In [0]:
%run ./helper

In [0]:
# ============================= Global Variable Declaration ===================================#
from pyspark.sql.functions import (
    col,
    concat,
    lit,
    current_date,
    lag,
    lead,
    first,
    last,
    desc,
    hash,
    row_number,
)
from datetime import date, timedelta, datetime
from pyspark.sql import functions as F
from pyspark.sql import *
from delta.tables import *
from pyspark.sql.types import TimestampType, LongType, StringType

# from ibi_packages import functions as fn
import os
import stringcase
from pyspark.sql import SQLContext
from pyspark.sql.window import Window


In [0]:
#================= Load BI config parameters data (overwrite) =================#W
#static variables
environment_name = environment
filename = "automlrevfoecastconfig.json"

today = pd.to_datetime('today').normalize()
current_eom = today + pd.offsets.MonthEnd(0)
start_date = '2017-01-01'
end_date = current_eom + pd.offsets.MonthEnd(11)

notebook = os.path.basename(getNotebookPath())

input_data_path = "/dbfs/mnt/"+ environment + "/automl_rev_region_forecast/inputs/"
output_data_path = "/dbfs/mnt/"+ environment + "/automl_rev_region_forecast/outputs/"
blobstore_datadir = "revregionforecast_data/"

bi_config_parameter_filepath =  "/mnt/{}/automl_rev_region_forecast/config/{}".format(environment_name,filename)

try:
    # read JSON file
    df_bi_config_parameters = (
        spark.read.format("json")
        .option("multiline", "true")
        .load(bi_config_parameter_filepath)
    )
    df_bi_config_parameters = df_bi_config_parameters.filter(
        df_bi_config_parameters.SystemName == lit(system_name)
    )
    print(df_bi_config_parameters)

    # adding audit fields
    # df_bi_config_parameters = df_bi_config_parameters.withColumn("IBICreatedBy",lit(ibi_created_by))
    # df_bi_config_parameters = df_bi_config_parameters.withColumn("IBIUpdatedBy",lit(ibi_updated_by))
    # df_bi_config_parameters = df_bi_config_parameters.withColumn("IBICreatedDate",lit(ibi_created_date).cast(TimestampType()))
    # df_bi_config_parameters = df_bi_config_parameters.withColumn("IBIUpdatedDate",lit(ibi_updated_date).cast(TimestampType()))
    # df_bi_config_parameters = df_bi_config_parameters.withColumn('ID', row_number().over(Window.orderBy('EnvironmentName','SystemName')))

    # initializing config parameter values

    # subscription_id = "db61fd47-db56-45e3-844f-1b1f5c47990a" #you should be owner or contributor
    subscription_id = dbutils.secrets.get(
        scope="kv-bi-devqa-01-secrets", key="subscription-id"
    )

    resource_group = (
        df_bi_config_parameters.filter(
            df_bi_config_parameters.ParameterName == "ws_resource_group"
        )
        .select("ParameterValue")
        .collect()[0][0]
    )
    workspace_name = (
        df_bi_config_parameters.filter(
            df_bi_config_parameters.ParameterName == "workspace_name"
        )
        .select("ParameterValue")
        .collect()[0][0]
    )
    workspace_region = (
        df_bi_config_parameters.filter(
            df_bi_config_parameters.ParameterName == "workspace_region"
        )
        .select("ParameterValue")
        .collect()[0][0]
    )


except Exception as error:
    log_error("{} {}".format(notebook, error))



In [0]:
# Create Workspace if required
'''
from azureml.core import Workspace

ws = Workspace.create(name = workspace_name,
                      subscription_id = subscription_id,
                      resource_group = resource_group, 
                      location = workspace_region,                      
                      exist_ok=True)
ws.get_details()

'''

In [0]:
# Import the Workspace class and configure your local envionment
from azureml.core import Workspace

# from azureml.core.authentication import InteractiveLoginAuthentication
import os
from azureml.core.authentication import ServicePrincipalAuthentication

tenant_id = "cfa930ff-a3f0-4933-8cea-744f8ead7682"
service_principal_id = "626c2c00-26de-46ee-a20d-6027445518ff"

svc_pr_password = dbutils.secrets.get(
    scope="kv-bi-devqa-01-secrets", key="dbw-azureml-devqa-key"
)

svc_pr = ServicePrincipalAuthentication(
    tenant_id=tenant_id,
    service_principal_id=service_principal_id,
    service_principal_password=svc_pr_password,
)


try:
    ws = Workspace(
        workspace_name=workspace_name,
        subscription_id=subscription_id,
        resource_group=resource_group,
        auth=svc_pr,  # forced_interactive_auth
    )

    # Persist the subscription id, resource group name, and workspace name in aml_config/config.json.
    ws.write_config()
    print(ws)
    print("Found workspace {} at location {}".format(ws.name, ws.location))
    # ws.write_config(path="./aml_config/",file_name="ws_config.json")
    # use the get method to load an existing workspace without using configuration files.
except Exception as error:
    log_error("{} {}".format(notebook, error))


In [0]:
# Create an experiement
import logging
import os
import random
import time
import json

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

# https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/automated-machine-learning/forecasting-orange-juice-sales/auto-ml-forecasting-orange-juice-sales.ipynb


In [0]:
# Choose a name for the experiment and specify the project folder.
experiment_name = 'automl-revenue-region-forecast'

try:

  experiment = Experiment(ws, experiment_name)

  output = {}
  output['SDK version'] = azureml.core.VERSION
  output['Subscription ID'] = ws.subscription_id
  output['Workspace Name'] = ws.name
  output['Resource Group'] = ws.resource_group
  output['Location'] = ws.location
  output['Experiment Name'] = experiment.name
  pd.set_option('display.max_colwidth', -1)
  pd.DataFrame(data = output, index = ['']).T
except Exception as error:
  log_error("{} {}".format(notebook, error))


In [0]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU clustera
try:
    amlcompute_cluster_name = aml_compute_cluster_name

    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster, use it:", amlcompute_cluster_name)
except Exception as error:
    log_error("{} {}".format(notebook, error))


In [0]:
# download SP500 closing prices
# if you get an error after executing the code, try adding below.
'''

#!pip install pandas_datareader
import pandas_datareader.data as web
import pandas_datareader as pdr

import datetime

pd.set_option('display.float_format', lambda x: '%.2f' % x)

start = datetime.datetime(2016, 1, 1)
end = today

# pd.core.common.is_list_like = pd.api.types.is_list_like

# SP500 = web.DataReader(['sp500'], 'fred', start, end)
# display(SP500.head())
try: 
  SP500 = pdr.get_data_fred('sp500', start, end)
  SP500 = SP500.reset_index(level=SP500.index.names)
  SP500['End_of_Month'] = SP500['DATE'] + pd.offsets.MonthEnd(0)
  SP500 = SP500.sort_values('DATE').groupby('End_of_Month').last()

  # SP500['Monthly_Return'] = (SP500['sp500']/ SP500['sp500'].shift(1)) -1
  # SP500['sp500_1M_Previous'] = SP500['sp500'].shift(1)

  SP500 = SP500.drop('DATE', axis=1)
  SP500 = SP500.reset_index(level=SP500.index.names)

  #Drop all Not a number values using drop method.
  SP500.dropna(inplace = True)
except Exception as error:
  log_error("{} {}".format(notebook, error))

print(' SP500: ')
print(get_df_name(SP500))
print(' DF Info: ')
print(SP500.info(verbose=True))
print(' SP500 Describe: ')
print(SP500.describe(include='all').transpose().head())
print(' SP500 Head: ')
print(SP500.head())
print(' SP500 Tail: ')

display(SP500.tail())
SP500['sp500'].plot(title='S&P 500')
'''

In [0]:
# download adjusted closing prices from Yahoo finance
#!pip install pandas_datareader
#!pip install yfinance
import pandas as pd
import yfinance as yf
import datetime
import time
import requests
import io

pd.set_option('display.float_format', lambda x: '%.2f' % x)

start = datetime.datetime(2016, 1, 1)
end = today

# Define the instruments to download. We would like to see Apple, Microsoft and the S&P500 index.
symbols  = ['^GSPC','^FTSE','^N100','^N225','000001.SS',]
symbols_name = ['SP_500','FTSE_100','EURONEXT_100','Nikkei_225','SSE_Composite_Index']


# create empty dataframe
stock_final = pd.DataFrame()
# iterate over each symbol
for i in symbols:  
    
    # print the symbol which is being downloaded
    print( str(symbols.index(i)) + str(' : ') + i, sep=',', end=',', flush=True)  
    
    try:
        # download the stock price 
        stock = []
        stock = yf.download(i,start=start, end=end, progress=False)
        
        # append the individual stock prices 
        if len(stock) == 0:
            None
        else:
            stock['Name']=i
            stock_final = stock_final.append(stock,sort=False)
    except Exception:
        None

#display(stock_final.tail())

stock_final1 = stock_final.copy()
stock_final1 = stock_final1.reset_index(level=stock_final1.index.names)

stock_final1["Name"].where(stock_final1["Name"] != "^N225", "Nikkei_225", inplace=True)
stock_final1["Name"].where(
    stock_final1["Name"] != "000001.SS", "SSE_Composite_Index", inplace=True
)
stock_final1["Name"].where(stock_final1["Name"] != "^GSPC", "SP_500", inplace=True)
stock_final1["Name"].where(stock_final1["Name"] != "^FTSE", "FTSE_100", inplace=True)
stock_final1["Name"].where(
    stock_final1["Name"] != "^N100", "EURONEXT_100", inplace=True
)

stock_final1['End_of_Month'] = stock_final1['Date'] + pd.offsets.MonthEnd(0)

# display(stock_final1.tail())
stock_final1 = stock_final1.sort_values("Date").groupby(["End_of_Month", "Name"]).last()
stock_final1 = stock_final1.drop(
    ["Date", "Open", "High", "Low", "Close", "Volume"], axis=1
)
stock_final1 = stock_final1.reset_index(level=stock_final1.index.names)

# Drop all Not a number values using drop method.
stock_final1.dropna(inplace=True)
stock_final_pivot = stock_final1.pivot_table(
    index="End_of_Month", values="Adj Close", margins=False, columns=["Name"]
)
# aggfunc=['sum','count'],

# revhist_pivot = revhist_pivot.rename(columns={'sum':'Revenue'})

stock_final_pivot = stock_final_pivot.reset_index(
    level=stock_final_pivot.index.names
).reset_index(drop=True)

# revhist_pivot['End_of_Month'] = pd.to_datetime(revhist_pivot['End_of_Month'])#.dt.date
# revhist_pivot.index.name = 'Date'

stock_final_pivot.fillna(0, inplace=True)
# stock_final1.query('Name=="Nikkei 225"').plot(title=str(i))
display(stock_final_pivot.info())
display(stock_final_pivot.tail())
for i in symbols_name:
    stock_final_pivot[i].plot(legend=True)

In [0]:
# Load Power BI Data into pandas Dataframe
# https://stbidatadevqa01.dfs.core.windows.net/dev/automl_rev_region_forecast


'''
if not os.path.isdir(input_data_path):
    os.mkdir(input_data_path, exist_ok=True)
if not os.path.isdir(output_data_path):
    os.mkdir(output_data_path, exist_ok=True)    
#os.makedirs('./inputs', exist_ok=True)
#os.makedirs('./outputs', exist_ok=True)
'''
# Ingest power bi data
init_time_column_name = 'End of Month'
init_time_column_name1 = 'Snapshot Date Short'
init_time_column_name2 = 'Snapshot End of Month'
'''
talenthist = spark.read.format("csv").load(input_data_path + 'TalentHistory.csv',header = True, quote = '"',multiLine = True, sep = "," , escape = '"')
revhist = spark.read.format("csv").load(input_data_path + 'RevenueHistory.csv',header = True, quote = '"',multiLine = True, sep = "," , escape = '"')
opphist = spark.read.format("csv").load(input_data_path + 'OppHistory.csv',header = True, quote = '"',multiLine = True, sep = "," , escape = '"')
projhist = spark.read.format("csv").load(input_data_path + 'ProjectHistory.csv',header = True, quote = '"',multiLine = True, sep = "," , escape = '"')
pipetrend = spark.read.format("csv").load(input_data_path + 'PipelineTrend.csv',header = True, quote = '"',multiLine = True, sep = "," , escape = '"')
pipehist = spark.read.format("csv").load(input_data_path + 'PipelineHistory.csv',header = True, quote = '"',multiLine = True, sep = "," , escape = '"')

'''
try:
  talenthist = pd.read_csv(input_data_path + 'TalentHistory.csv', parse_dates=[init_time_column_name])
  revhist = pd.read_csv(input_data_path + 'RevenueHistory.csv', parse_dates=[init_time_column_name])
  opphist = pd.read_csv(input_data_path + 'OppHistory.csv', parse_dates=[init_time_column_name])                      
  projhist = pd.read_csv(input_data_path + 'ProjectHistory.csv', parse_dates=[init_time_column_name])
  pipetrend = pd.read_csv(input_data_path + 'PipelineTrend.csv', parse_dates=[init_time_column_name, init_time_column_name1])
  pipehist = pd.read_csv(input_data_path + 'PipelineHistory.csv', parse_dates=[init_time_column_name, init_time_column_name1])
except Exception as error:
  log_error("{} {}".format(notebook, error))

for x in [pipehist, talenthist, revhist, opphist, projhist, pipetrend]:
    # replace blank with underscore
    x.columns = x.columns.astype(str).str.replace(" ", "_")

    # remove contrib or managing
    x.columns = x.columns.astype(str).str.replace("Contrib_", "")
    x.columns = x.columns.astype(str).str.replace("Managing_", "")

    print(get_df_name(x), ":", x.shape)
    display(x.info())
    display(x.tail())


In [0]:
# Perform transformations

# set variales
target_column_name = "Revenue"
time_column_name = "End_of_Month"
time_series_id_column_names = ["Relative_Offset", "Sub_Region_Code"]
sort_cols_snp = ["End_of_Month", "Snapshot_Date", "Sub_Region_Code"]
sort_cols_eom = ["End_of_Month", "Sub_Region_Code"]

exclude_sub_region = ["Singapore", "NA"]

pd.set_option("display.float_format", lambda x: "%.2f" % x)

talenthist1 = talenthist.copy()  # .toPandas() to convert from spark df to pandas
pipehist1 = pipehist.copy()
opphist1 = opphist.copy()
projhist1 = projhist.copy()
revhist1 = revhist.copy()
pipetrend1 = pipetrend.copy()

pipetrend1.drop(columns="Snapshot_Date_Short", axis=1, inplace=True)
pipehist1.rename(columns={"Snapshot_Date_Short": "Snapshot_Date"}, inplace=True)
talenthist1['Billable_Headcount'] = (talenthist1['Headcount'] + talenthist1['Headcount_Contingent'])
talenthist1.drop(columns=["Headcount", "Headcount_Contingent"], axis=1, inplace=True)


# Remove Percentage from numbers if exist
cols_float1 = opphist1.filter(like="Rate", axis=1).columns
remove_percetage(opphist1, cols_float1)
cols_float2 = pipehist1.filter(like="Yield", axis=1).columns
remove_percetage(pipehist1, cols_float2)

for x in [pipehist1, talenthist1, revhist1, opphist1, projhist1, pipetrend1]:
    x.infer_objects()

    # convert date to to_datetime
    x = convert_date_cols(x)

    # coerce some fields to numeric
    if "Pipeline" in x.columns:
        x["Pipeline"] = (
            x["Pipeline"]
            .apply(pd.to_numeric, errors="coerce")
            .replace(np.nan, 0, regex=True)
        )
    if "Revenue" in x.columns:
        x["Revenue"] = (
            x["Revenue"]
            .apply(pd.to_numeric, errors="coerce")
            .replace(np.nan, 0, regex=True)
        )
    if "Conversions" in x.columns:
        x["Conversions"] = (
            x["Conversions"]
            .apply(pd.to_numeric, errors="coerce")
            .replace(np.nan, 0, regex=True)
        )
    # replace null values to NA
    x["Sub_Region_Code"] = x["Sub_Region_Code"].replace(np.nan, "NA", regex=True)

    x = x.replace(np.nan, 0, regex=True)

    # make all dates to End of Month values to later merge

    x['End_of_Month'] = x['End_of_Month'] + pd.offsets.MonthEnd(0)
  
    if 'Snapshot_Date' in x.columns:
      x['Snapshot_Date'] = x['Snapshot_Date'] + pd.offsets.MonthEnd(0)
      x.sort_values(by=sort_cols_snp).reset_index(drop=True)
    else:
      x.sort_values(by=sort_cols_eom).reset_index(drop=True)

    print(get_df_name(x), ":", x.shape)
    display(x.info())
    display(x.tail())

In [0]:
# Cross join to get all unquie values of End_of_Month and Studio

#Set Parameters---------------------------------------------------------------------------------------------------------------------
relative_offset_past_period = 0
n_test_periods = 4


d = pd.date_range(start_date, today + pd.offsets.MonthEnd(n_test_periods) , freq='m')
eom = pd.DataFrame(d, columns=["End_of_Month"])

# eom = pipehist_pivot1[['End_of_Month']].drop_duplicates(subset='End_of_Month',keep='last').sort_values(by=['End_of_Month']).reset_index(drop=True)
sub_region = pd.DataFrame(
    pipehist1["Sub_Region_Code"].unique(), columns=["Sub_Region_Code"]
)
snp_date = pd.DataFrame(pipehist1["Snapshot_Date"].unique(), columns=["Snapshot_Date"])

eom["key"] = 0
snp_date["key"] = 0
sub_region["key"] = 0

eom1 = pd.merge(eom, snp_date, on=["key"]).drop("key", 1)
eom1["key"] = 0
eom2 = pd.merge(eom1, sub_region, on=["key"]).drop("key", 1)
eom_region_snp = eom2.replace(np.nan, "NA", regex=True).query(
    "Sub_Region_Code not in @exclude_sub_region"
)
# eom_region_snp.drop(columns=['key'], axis=1, inplace = True)

# Add offset columns
eom_region_snp["Relative_Month_Offset"] = round(
    (eom_region_snp["End_of_Month"] - current_eom) / np.timedelta64(1, "M"), 0
).astype(int)
eom_region_snp["Relative_Snapshot_Month_Offset"] = round(
    (eom_region_snp["Snapshot_Date"] + pd.offsets.MonthEnd(0) - current_eom) / np.timedelta64(1, "M"), 0
).astype(int)
eom_region_snp["Relative_Offset"] = (
    eom_region_snp["Relative_Month_Offset"]
    - eom_region_snp["Relative_Snapshot_Month_Offset"]
)
eom_region_snp["Relative_Offset"] = eom_region_snp["Relative_Offset"].astype(int)

# FILTER for relevant history and forecast period rows
eom_region_snp = eom_region_snp.query(
    "Relative_Offset >= -@relative_offset_past_period and Relative_Offset < @n_test_periods and Relative_Snapshot_Month_Offset <= 0"
)
cols = ['Relative_Offset','Relative_Snapshot_Month_Offset','Relative_Month_Offset']
eom_region_snp[cols] = eom_region_snp[cols].apply(pd.to_numeric, errors="coerce")
# display(eom.info(verbose=True))
print("EOM :", eom.shape)
print("Region Shape:", sub_region.shape)
print("Snapthot Shape:", snp_date.shape)
print("EOM_Region Shape:", eom_region_snp.shape)
eom_region_snp


In [0]:

eom_region_snp_pvt = eom_region_snp.pivot_table(
    index=[time_column_name],
    columns="Sub_Region_Code",
    values=["Snapshot_Date"],
    aggfunc={"Snapshot_Date": "count"},
    margins=False,
)
# eom_region_snp_pvt = eom_region_snp_pvt.reset_index(level=eom_region_snp_pvt.index.names).reset_index(drop=True)
eom_region_snp_pvt.head()

In [0]:
eom_region_snp_pvt.tail()

In [0]:
# Merge eom_studio with Revenue, Talent, Opportunity, Project

from functools import reduce
eom_region_snp1 = eom_region_snp.copy()

nan_value = 0
merge_dfs = [eom_region_snp1, revhist1, talenthist1, opphist1, projhist1, pipetrend1]

merge_df = reduce(
    lambda left, right: pd.merge(
        left, right, how="left", on=["End_of_Month", "Sub_Region_Code"]
    ),
    merge_dfs,
).fillna(nan_value)
coerce_df_columns_to_numeric(merge_df)
merge_df = merge_df.replace(np.nan, 0, regex=True)
merge_df.sort_values(by=sort_cols_snp).reset_index(drop=True)

# merge_df.drop(columns=['Current_Opp._Period_Value','Conversions'], axis=1, inplace = True)

print(" merge_df: ", merge_df.shape)

# merge_df.to_csv(output_data_path   'merge_df.csv', index=False)
# show_stats(merge_df)
merge_df.info()
print(merge_df.shape)
merge_df.tail()  # .query('Fin_Entity_ID==@entity_debug')')

In [0]:
# Merge with SP500
'''
eom1 = eom  # pd.DataFrame(d, columns=['End_of_Month'])
sp500_df = pd.merge(eom1, SP500, how='left', on=['End_of_Month'])

# sp500_df = sp500_df.ffill(axis = 0)

sp500_df = sp500_df.sort_values(by=sort_cols_eom).reset_index(drop=True)
# display(sp500_df.tail(10))
merge_df1 = pd.merge(merge_df, sp500_df, how='left', on=['End_of_Month'
                     ])
coerce_df_columns_to_numeric(merge_df1)
merge_df1.drop(columns=['Headcount', 'Headcount_Contingent'], axis=1,
               inplace=True)  # ,'Total_Headcount','Monthly_Return','sp500_1M_Previous'
merge_df1 = merge_df1.sort_values(by=sort_cols_eom).reset_index(drop=True)
merge_df1.to_csv(output_data_path + 'merge_df1.csv', index=False)
#merge_sdf1 = spark.createDataFrame(merge_df1);
#merge_sdf1.write.format("parquet").mode("overwrite").parquet(output_data_path + 'merge_df1.parquet')

# show_stats(merge_df1.tail(10))
# merge_df1.info()
merge_df1.tail(10)
'''

In [0]:
# Merge with eom_region_snp1 with stock_final_pivot

#eom1 = eom  # pd.DataFrame(d, columns=['End_of_Month'])
#stock_final_df = pd.merge(
#    eom_region_snp1, stock_final_pivot, how="left", on=["End_of_Month","Relative_Month_Offset","Relative_Snapshot_Month_Offset","Relative_Offset"]
#).fillna(nan_value)

# sp500_df = sp500_df.ffill(axis = 0)

#stock_final_df = stock_final_df.sort_values(by=sort_cols_eom).reset_index(drop=True)
# display(sp500_df.tail(10))
merge_df1 = pd.merge(merge_df, stock_final_pivot, how="left", on='End_of_Month').fillna(nan_value)
coerce_df_columns_to_numeric(merge_df1)
merge_df1 = merge_df1.sort_values(by=sort_cols_snp).reset_index(drop=True)
# merge_df1.to_csv(output_data_path   'merge_df1.csv', index=False)
# merge_sdf1 = spark.createDataFrame(merge_df1);
# merge_sdf1.write.format("parquet").mode("overwrite").parquet(output_data_path   'merge_df1.parquet')

# show_stats(merge_df1.tail(10))
# merge_df1.info()
print(merge_df1.shape)
merge_df1.info()
merge_df1.tail()


In [0]:
# Calculate pipeline month offset for corresponding snapshot dates

# Set Parameters---------------------------------------------------------------------------------------------------------------------
pipehist_long = pipehist1.copy()
# pipehist2 = pipehist2.query('Relative_Offset>=-11 and Relative_Offset<=0')
# pipehist_long['Relative_Offset'] = pipehist_long['Relative_Offset'].astype(str).str.zfill(2)   'M Pipeline'
# pipehist_long['Relative_Offset'] = pipehist_long['Relative_Offset'].str.replace('0',' ')
# pipehist_long['Relative_Offset'] = pipehist_long['Relative_Offset'].str.replace('  M Pipeline',' 0M Pipeline')
# pipehist_long['Relative_Offset'] = 'Relative'   pipehist_long['Relative_Offset'].str.replace('0-','-0')

# pipehist_long.sort_values(by=sort_cols_snp).reset_index(drop=True)  # ,'Fin_Entity_ID'

# print (' pipehist_long: ')

# pipehist_long.to_csv(output_data_path   'pipehist_long.csv', index=False)
# display(pipehist_long.tail(10))  # .query('Fin_Entity_ID==@entity_debug'))
pipehist_long.info()
print("Pipehist_long:", pipehist_long.shape)
pipehist_long.tail()
# merge_df1.tail()


In [0]:
# Pivot Pipeline_Type from long to wide to match month end grain
pipehist_wide = pipehist_long.pivot_table(
    index=[
        "Snapshot_Date",
        "Relative_Snapshot_Month_Offset",
        "End_of_Month",
        "Relative_Month_Offset",
        "Sub_Region_Code"
    ],
    columns="Pipeline_Type",
    values=["Pipeline"],  # , 'Pipeline_3M_Rolling_Avg'
    aggfunc={"Pipeline": np.sum},
    margins=False,
).fillna(nan_value)

pipehist_wide.columns = [
    "_".join(tuple(map(str, t))) for t in pipehist_wide.columns.values
]
pipehist_wide.columns = [
    "Active_Unrecognized",
    "Opportunity",
    "Opportunity_ML",
    "Recognized",
]
pipehist_wide = pipehist_wide.reset_index(level=pipehist_wide.index.names).reset_index(
    drop=True
)

# pipehist_wide.tail()
# pipehist_wide['Pipeline'] = pd.to_numeric(pipehist_wide['Pipeline'], errors='coerce').astype(int)
pipe_cols = ["Active_Unrecognized", "Opportunity", "Opportunity_ML", "Recognized"]
for x in pipe_cols:
    pipehist_wide[x] = (
        pd.to_numeric(pipehist_wide[x], errors="coerce")
        .replace(np.nan, 0, regex=True)
        .astype(int)
    )


#cols = list(pipehist_wide.select_dtypes(include="float64").columns)

#coerce_df_columns_to_numeric(pipehist_wide)

pipehist_wide["Snapshot_Date"] = pd.to_datetime(pipehist_wide["Snapshot_Date"])
pipehist_wide["End_of_Month"] = pd.to_datetime(pipehist_wide["End_of_Month"])

# pipehist_wide.rename(columns={0:'Pipeline_3M_Rolling_Avg'}, inplace=True)

pipehist_wide = pipehist_wide.sort_values(by=sort_cols_snp).reset_index(drop=True)

# print(' pipehist_wide: ')

display(pipehist_wide.info())

# pipehist_wide.query('Fin_Entity_ID==@entity_debug')
print("Pipehist_wide:", pipehist_wide.shape)
pipehist_wide.tail()  # .query('Relative_Snapshot_Month_Offset == 0')


In [0]:
"""
# Merge merge_df with Pipeline Long History
# pipe_offset = ['Relative-00','Relative-01','Relative-02','Relative-03',
#               'Relative-04','Relative-05','Relative-06'
#               ,'Relative-07', 'Relative-08','Relative-09','Relative-10','Relative-11'
#              ]
# 1merge_final = pd.merge(merge_df1, pipehist_long.query('Relative_Offset == 0'), how='left', on=['End_of_Month','Relative_Month_Offset','Fin_Entity_ID'])
# 1merge_final.drop(columns=['Snapshot_Date_Short','Relative_Snapshot_Month_Offset'], axis=1, inplace = True)
# New Code
"""

pipehist_wide1 = pipehist_wide.copy()
merge_final = pd.merge(
    merge_df1,
    pipehist_wide1,
    how="left",
    on=["End_of_Month", "Relative_Month_Offset", "Snapshot_Date", "Relative_Snapshot_Month_Offset", "Sub_Region_Code"],
).fillna(nan_value)
# print(merge_final.info())
# print(merge_final.tail())
# merge_final.drop(columns=['Snapshot_Date_Short','Relative_Snapshot_Month_Offset'], axis=1, inplace = True)
# New Code end

merge_final = merge_final.infer_objects()
merge_cols = [
    "Relative_Month_Offset",
    "Active_Unrecognized",
    "Opportunity",
    "Opportunity_ML",
    "Recognized",
    "Revenue",
    "Conversions",
    "Current_Opp._Period_Value",
    "Project_Period_Price",
    "Project_Price",
    "Pipeline_Trend",
    "Billable_Headcount",
]
for x in merge_cols:
    merge_final[x] = (
        pd.to_numeric(merge_final[x], errors="coerce")
        .replace(np.nan, 0, regex=True)
        .astype(int)
    )

# cond = ~merge_final['Snapshot_Date'].isna()
# merge_final['Snapshot_Date'] = np.where(cond,
#        merge_final['Snapshot_Date'], merge_final['End_of_Month'])
# merge_final['Relative_Snapshot_Month_Offset'] = \
#    round((merge_final['Snapshot_Date'] - current_eom)
#          / np.timedelta64(1, 'M'), 0).astype(int)

merge_final.fillna(0, inplace=True)


# Filter for only next 11 months rows w.r.t. to snapshot date
# revpipe_final1 = revpipe_final.query('Relative_Offset in @pipe_offset').reset_index(drop=True)
# merge_final = merge_final.replace(np.nan, 0, regex=True)
coerce_df_columns_to_numeric(merge_final)
merge_final.info()
merge_final = merge_final.sort_values(by=sort_cols_snp).reset_index(drop=True)
# display(merge_final.info())


merge_final.to_csv(output_data_path + 'merge_final.csv', index=False)
merge_final.to_parquet(output_data_path + 'merge_final.parquet', index=None)
'''
#merge_final_sdf = spark.createDataFrame(merge_final);
#merge_final_sdf.repartition(1).write.format("parquet").mode("overwrite").parquet(output_data_path + 'merge_final.parquet')
'''
print('merge_final:', merge_final.shape)
display(merge_final.info())
merge_final.tail()

#merge_final.query('Relative_Snapshot_Month_Offset == 0')
#show_stats(merge_final)


In [0]:
# Final check before spliting
sort_list = [time_column_name] + list(set(time_series_id_column_names))
df = merge_final.query('Sub_Region_Code != "NA"').copy()
df.replace(np.nan, 0)
# display(df.info(verbose=True))
df_pivot = df.pivot_table(
    index=[time_column_name],
    columns="Sub_Region_Code",
    values=["Relative_Offset"],
    aggfunc={"Relative_Offset": "count"},
    margins=False,
).fillna(nan_value)
df_pivot.T

In [0]:
df_pivot = df.pivot_table(
    index=[time_column_name],
    columns="Relative_Offset",
    values=["Sub_Region_Code"],
    aggfunc={"Sub_Region_Code": "count"},
    margins=False,
).fillna(nan_value)
df_pivot.T
# display(df.head())
# display(df.tail())

In [0]:
# Data Split
# split the data into a training and a testing set for later forecast evaluation. The test set will contain the final test_size months of observed sales for each time-series.
# The splits should be stratified by series, so we use a group-by statement on the time series identifier columns.
# test_size = .11 #in percentage
# n_test_periods =  int(tseries * test_size)
# n_test_periods = relative_offset_future_period

In [0]:
tseries = df.groupby(time_column_name).ngroups
print("Data contains {0} individual time_column_name.".format(tseries))
test_size = round(n_test_periods / tseries, 2)
print("Time Column Name:", time_column_name)
print("Test Size: ", test_size, "Test Periods: ", n_test_periods)
print(df[time_column_name].unique())


In [0]:
# df = merge_final.drop(columns=['End_of_Month','Relative_Month_Offset']).sort_values(by=sort_list)# axis=1, inplace = True)

nseries = df.groupby(time_series_id_column_names).ngroups
print("time_series_id_column_names:", time_series_id_column_names)
print("Data contains {0} individual time-series.".format(nseries))
print(sort_list)
print(df["Relative_Offset"].unique())
print(df["Sub_Region_Code"].unique())
print(df["End_of_Month"].unique())


In [0]:
#train, test = split_full_for_forecasting(df=df, time_column_name=time_column_name, grain_column_names=time_series_id_column_names, test_split=test_size)
print('n_test_periods: ', n_test_periods)
#(train, test) = split_last_n_by_series_id(df, n_test_periods)



train = df.query('Relative_Month_Offset < 0')
test = df.drop(train.index)
#test = merge_final1.query('(End_of_Month >= @current_eom)')#.sort_values(by=time_column_name)#.reset_index(drop=True)
#train = merge_final1.drop(test.index)#.sort_values(by=sort_list)#('(

print(df.shape)
print(train.shape)
print(test.shape)

# df1 = df.copy()
# train = df1.sample(frac=train_size, random_state=0)
# test = df1.drop(train.index)
# train_size = int(df.shape[0] * 0.9)
# train = df[:n_test_periods]
# test = df[n_test_periods:]
#train_sdf = spark.createDataFrame(train);
#test_sdf = spark.createDataFrame(test);
#train_sdf.repartition(1).write.format("csv").mode("overwrite").save(output_data_path + 'revforecast_train.csv', header = True)
#test_sdf.repartition(1).write.format("csv").mode("overwrite").save(output_data_path + 'revforecast_test.csv', header = True)
#train_sdf.repartition(1).write.format("parquet").mode("overwrite").parquet(output_data_path + 'revforecast_train.parquet')
#test_sdf.repartition(1).write.format("parquet").mode("overwrite").parquet(output_data_path + 'revforecast_test.parquet')
#'''

train.to_csv(output_data_path + 'revregionforecast_train.csv', index=None,
             header=True)
test.to_csv(output_data_path + 'revregionforecast_test.csv', index=None,
            header=True)
#train.to_parquet(output_data_path + 'revregionforecast_train.parquet',
#                       index=None)
#test.to_parquet(output_data_path + 'revregionforecast_test.parquet',
#                       index=None)            
#'''
# train_file_path = 'train.csv'
# test_file_path = 'eval.csv'

In [0]:
# Summary Stats for Train and Test dataframes

print ('Train************************')
print(train.info(verbose=True))
print(train.describe(include='all').transpose().head())
train.tail(10)

In [0]:
print ('Test************************')
print(test.info(verbose=True))
print(test.describe(include='all').transpose().head())
test.tail(10)

In [0]:
# plot the example time series
#!pip install seaborn
'''
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.float_format', lambda x: '%.4f' % x)

# sns.set_context('paper', font_scale=1.3)
# sns.set_style('white')

(fig, ax) = plt.subplots(figsize=(24, 9))
import matplotlib.pyplot as plt
whole_data = train.copy()
target_label = target_column_name
whole_data[target_label] = train[target_column_name]
for g in whole_data.groupby(time_series_id_column_names):
    plt.plot(g[1][time_column_name].values,
             g[1][target_column_name].values, label=g[0])
plt.legend()
#plt.show()
'''

In [0]:
# plot the example time series
'''
(fig, ax) = plt.subplots(figsize=(24, 9))
import matplotlib.pyplot as plt
whole_data = test.copy()
target_label = target_column_name
whole_data[target_label] = test[target_column_name]
for g in whole_data.groupby(time_series_id_column_names):
    plt.plot(g[1][time_column_name].values,
             g[1][target_column_name].values, label=g[0])
plt.legend()
#plt.show()
'''

In [0]:
from azureml.core import Dataset, Datastore

datastore = ws.get_default_datastore()
print("Default datastore's name: {}".format(datastore.name))
datastore.upload_files(files=[output_data_path + 'revregionforecast_train.csv',
                       output_data_path + 'revregionforecast_test.csv'],
                       target_path=blobstore_datadir, overwrite=True,
                       show_progress=True)

In [0]:
# Create dataset for training

#register_spark_dataframe(dataframe=train_sdf, target=blobstore_datadir, name="revforecast_train.parquet", description=None, tags=None, show_progress=True)

dataset_filename = blobstore_datadir + "revregionforecast_train.csv"
train_dataset = Dataset.Tabular.from_delimited_files(path=[(datastore, dataset_filename)])

display(train_dataset.to_pandas_dataframe().head())
display(train_dataset.to_pandas_dataframe().tail())


In [0]:
# Train

from azureml.automl.core.forecasting_parameters import ForecastingParameters

target_lag = "auto"  # [x for x in range(1,7)] #'auto'
window_size = "auto"
feature_lag = "auto"
forecast_horizon = n_test_periods

print("relative_offset_past_period", relative_offset_past_period)
print("n_test_periods", n_test_periods)
print("target_lags:", target_lag)
print("target_rolling_window_size:", window_size)
print("forecast_horizon:", forecast_horizon)
print("feature_lags:", feature_lag)

forecasting_parameters = ForecastingParameters(
    time_column_name=time_column_name,
    forecast_horizon=forecast_horizon,
    time_series_id_column_names=time_series_id_column_names,
    target_lags=target_lag,
    feature_lags=feature_lag,
    target_rolling_window_size=window_size,
)

# drop_column_names = ['-1M Pipeline']

automl_config = AutoMLConfig(  # featurization_config,
    task="forecasting",
    debug_log="rev_region_forecast_errors.log",
    primary_metric="normalized_root_mean_squared_error",
    experiment_timeout_hours=4,
    training_data=train_dataset,
    label_column_name=target_column_name,
    # spark_context=sc, #enable this for databricks cluster
    compute_target=compute_target,  # enable this for ml cluster
    enable_early_stopping=True,
    enable_dnn=True,  # enable this for ml cluster
    featurization="auto",
    n_cross_validations=5,
    verbosity=logging.INFO,
    max_concurrent_iterations=8,
    max_cores_per_iteration=-1,
    forecasting_parameters=forecasting_parameters,
)


In [0]:
# submit a new training run
try:
    if new_training == "True":
        print("New Training Run")
        remote_run = experiment.submit(automl_config, show_output=True)
        remote_run
    else:
        # If you need to retrieve a run that already started, use the following code
        print("Existing Training Run")
        from azureml.train.automl.run import AutoMLRun

        remote_run = AutoMLRun(
            experiment=experiment, run_id="AutoML_43e55a9b-17a1-4352-a247-3e9df6738656"
        )
except Exception as error:
    log_error("{} {}".format(notebook, error))


In [0]:
rr = remote_run.wait_for_completion()
run_id = rr.get('runId')
run_datetime = rr.get('endTimeUtc')
print(run_id, run_datetime)

In [0]:
# Retrieve the Best Model

(best_run, fitted_model) = remote_run.get_output()
print ('Best Run Model: ', best_run)
print (fitted_model.steps)
model_name = best_run.properties['model_name']
#print ('Model Name: ', model_name)

In [0]:
# Transparency
# View updated featurization summary

featurization_summary = fitted_model.named_steps[
    "timeseriestransformer"
].get_featurization_summary()

# View the featurization summary as a pandas dataframe

pd.DataFrame.from_records(featurization_summary)


In [0]:
# Explore the results
'''
from azureml.widgets import RunDetails
RunDetails(best_run).show()
'''
#displayHTML("<a href={} target='_blank'>Azure Portal: {}</a>".format(best_run.get_portal_url(), best_run.id))

In [0]:
# Get metrics for best run
pd.set_option("display.float_format", lambda x: "%.5f" % x)
metricslist = {}
properties = best_run.get_properties()
# print(properties)
metrics = {k: v for k, v in best_run.get_metrics().items() if isinstance(v, float)}
metricslist[int(properties["iteration"])] = metrics
rundata = pd.DataFrame(metricslist).sort_index(1)
rundata


In [0]:
# FORECASTING

X_test = test.copy()
y_test = X_test.pop(target_column_name).values

# forecast returns the predictions and the featurized data, aligned to X_test.
# This contains the assumptions that were made in the forecast
# The featurized data, aligned to y, will also be returned.
# This contains the assumptions that were made in the forecast
# and helps align the forecast to the original data

y_predictions, X_trans = fitted_model.forecast(X_test)

# from forecasting_helper import align_outputs

df_all = align_outputs(y_predictions, X_trans, X_test, y_test, target_column_name)
df_all.rename(columns={"predicted": "Predicted_Revenue"}, inplace=True)

df_all.tail(10)


In [0]:
# Evaluate
# To evaluate the accuracy of the forecast, we'll compare against the actual sales quantities for some select metrics, included the mean absolute percentage error (MAPE).

# We'll add predictions and actuals into a single dataframe for convenience in calculating the metrics.
'''
assign_dict = {'Predicted_Revenue': y_predictions, target_column_name: y_test}
df_all = X_test.assign(**assign_dict)
df_all.tail(10)
'''

In [0]:
from azureml.automl.core.shared import constants
from azureml.automl.runtime.shared.score import scoring
from matplotlib import pyplot as plt

# use automl scoring module

scores = scoring.score_regression(
    y_test=df_all[target_column_name],
    y_pred=df_all["Predicted_Revenue"],
    metrics=list(constants.Metric.SCALAR_REGRESSION_SET),
)

print("[Test data scores]")
for key, value in scores.items():
    print("{}:   {:.3f}".format(key, value))

# Plot outputs

%matplotlib inline
test_pred = plt.scatter(df_all[target_column_name], df_all["Predicted_Revenue"], color="b")
test_test = plt.scatter(
    df_all[target_column_name], df_all[target_column_name], color="g"
)
plt.legend(
    (test_pred, test_test), ("prediction", "truth"), loc="upper left", fontsize=8
)
plt.show()

In [0]:
# Print metrics by different groups

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
'''
if "horizon_origin" in df_all.columns:
    group_by = "horizon_origin"
else:
'''    
group_by = 'Relative_Offset'

df_mpae = df_all.groupby(group_by).apply(
    lambda df: pd.Series(
        {
            "MAPE": MAPE(df[target_column_name], df["Predicted_Revenue"]),
            "RMSE": np.sqrt(
                mean_squared_error(df[target_column_name], df["Predicted_Revenue"])
            ),
            "MAE": mean_absolute_error(df[target_column_name], df["Predicted_Revenue"]),
            "R2_SCORE": r2_score(df[target_column_name], df["Predicted_Revenue"])
            
        }
    )
)
print(df_mpae)

# Box plot with different groups
df_all_APE = df_all.assign(APE=APE(df_all[target_column_name], df_all['Predicted_Revenue']))
'''
if 'horizon_origin' in df_all_APE.columns:
  APEs = [df_all_APE[df_all['horizon_origin'] == h].APE.values for h in range(1, forecast_horizon + 1)]
  group_by = "horizon_origin"
else:
'''
APEs = [df_all_APE[df_all[group_by] == h].APE.values for h in range(1, forecast_horizon + 1)]


%matplotlib inline
plt.boxplot(APEs)
plt.yscale('log')
plt.xlabel(group_by)
plt.ylabel('APE (%)')
plt.title('Absolute Percentage Errors by Forecast Horizon')

plt.show()

In [0]:
# Confidence intervals

quantiles =  fitted_model.forecast_quantiles(X_test)
#quantiles.tail(10)

# specify which quantiles you would like 
'''
fitted_model.quantiles = [0.01, 0.5, 0.8, 0.95]

# use forecast_quantiles function, not the forecast() one

y_pred_quantiles =  fitted_model.forecast_quantiles(X_test)

# quantile forecasts returned in a Dataframe along with the time and time series id columns 

#y_pred_quantiles.tail(10)
'''

In [0]:
# Merge df_all with train

from functools import reduce

# Get the column index list in the right order

cols_sort_list = list(df_all.columns) + ['Revenue_Forecast']
print(cols_sort_list)
#df_all.tail(10)

nan_value = 0
merge_dfs = [train, df_all]

final_merge_df = train.append(df_all, ignore_index=True,
                              sort=False).sort_values(by=sort_cols_snp)
coerce_df_columns_to_numeric(final_merge_df)

final_merge_df = final_merge_df.replace(np.nan, 0, regex=True)

# merge_df = merge_df.query('Fin_Entity_ID!='SGP'') # Exclude Singapore rows
# final_merge_df.sort_values(by=['Relative_Offset','Fin_Entity_ID']).reset_index(drop=True) and Relative_Month_Offset >= 0
# merge_df.drop(columns=['Current_Opp._Period_Value','Conversions'], axis=1, inplace = True)
# create new column

final_merge_df['Revenue_Forecast'] = \
    np.where(final_merge_df['Predicted_Revenue'] != 0,
             final_merge_df['Predicted_Revenue'],
             final_merge_df['Revenue'])
final_merge_df['Relative_Month_Offset'] = \
    round((final_merge_df['End_of_Month'] - current_eom)
          / np.timedelta64(1, 'M'), 0).astype(int)
final_merge_df['End_of_Month'] = \
    pd.to_datetime(final_merge_df['End_of_Month'])  # .dt.date

if 'origin' in final_merge_df.columns:
    final_merge_df['origin'] = pd.to_datetime(final_merge_df['origin'])

            # .dt.date

final_merge_df['Snapshot_Date'] = \
    pd.to_datetime(final_merge_df['Snapshot_Date'])  # .dt.date



# sort using original cols_sort_list

final_merge_df = \
    final_merge_df.sort_values(by=sort_cols_snp).reset_index(drop=True)
final_merge_df = final_merge_df.reindex(columns=cols_sort_list)

# add revenue vs Predicted_Revenue Predicted_Revenue_Variance

final_merge_df['Predicted_Revenue_Variance'] = \
    np.where(final_merge_df['Relative_Month_Offset'] < 0,
             final_merge_df['Revenue']
             - final_merge_df['Predicted_Revenue'], np.nan)
final_merge_df['Predicted_Revenue_Variance_Percent'] = \
    np.where(final_merge_df['Relative_Month_Offset'] < 0,
             round(final_merge_df['Predicted_Revenue_Variance']
             / final_merge_df['Predicted_Revenue'] * 100, ndigits=4),
             np.nan)
final_merge_df['Predicted_Revenue_Variance'].where(final_merge_df['Predicted_Revenue']
                                 != 0, np.nan, inplace=True)
final_merge_df['Predicted_Revenue_Variance_Percent'
               ].where(final_merge_df['Predicted_Revenue'] != 0,
                       np.nan, inplace=True)
final_merge_df['Predicted_Revenue_Variance_Percent'] = \
    round(final_merge_df['Predicted_Revenue_Variance_Percent'], ndigits=4)
final_merge_df['Run_ID'] = run_id
final_merge_df['Forecast_Date'] = \
    pd.to_datetime(run_datetime).normalize().date()
final_merge_df['Forecast_Date'] = \
    pd.to_datetime(final_merge_df['Forecast_Date'])  # .dt.date
final_merge_df = convert_date_cols(final_merge_df)

print (' final_merge_df: ')
final_merge_df.info()
final_merge_df.to_csv(output_data_path + 'final_merge_df.csv',
                      index=False)
final_merge_df.to_parquet(output_data_path + 'final_merge_df.parquet',
                          index=None)

# final_merge_df
# display(final_merge_df.tail(28))
# final_merge_df  # .query('Fin_Entity_ID==@entity_debug')


In [0]:
# pivot by EOM1, Fin_Entity_ID
pd.set_option("display.float_format", lambda x: "%.2f" % x)  #

final_merge_df1 = final_merge_df.copy()
cols = [
    "Predicted_Revenue",
    "Revenue",
    "Predicted_Revenue_Variance",
    "Predicted_Revenue_Variance_Percent",
]
#final_cond = "Relative_Offset >= 0 and Relative_Snapshot_Month_Offset <= -3"
#final_merge_df1 = final_merge_df.query(final_cond)

final_merge_df1 = convert_date_cols(final_merge_df1)

final_merge_pivot1 = final_merge_df1.pivot_table(
    index=[
        "End_of_Month","Relative_Month_Offset",
        "Snapshot_Date","Relative_Snapshot_Month_Offset",
        "Relative_Offset"
    ],  
    values=cols,
    aggfunc={
        "Predicted_Revenue": np.sum,
        "Revenue": np.sum,
        "Predicted_Revenue_Variance": np.sum,
        "Predicted_Revenue_Variance_Percent": np.mean,
    },
    margins=None,
).fillna(nan_value)
# final_merge_pivot1['End_of_Month'] = pd.to_datetime(final_merge_pivot1['End_of_Month']).dt.date
# final_merge_pivot1.reset_index(level=final_merge_pivot1.index.names)

# show_stats(revtime_pivot1)
final_merge_pivot1.sort_values(by=['End_of_Month','Snapshot_Date','Relative_Offset'], inplace=True)
# reorder columns
# cols_order = [0,4,1,2,3]
# final_merge_pivot1 = final_merge_pivot1[[final_merge_pivot1.columns[i] for i in cols_order]]

# set ALL float columns to '${:,.2f}' formatting (including the percentage)
# format_dict = {col_name: '${:,.1f}' for col_name in final_merge_pivot1.select_dtypes(float).columns}
# override the percentage column
format_dict = {col_name: "{:,}" for col_name in final_merge_pivot1.columns}
format_dict["Predicted_Revenue_Variance_Percent"] = "{:.2f}"


# Format with commas and round off to two decimal places in pandas
# final_merge_pivot1 = final_merge_pivot1[[['Snapshot_Date_Short','End_of_Month'] cols]]

In [0]:
final_merge_pivot1.query('Relative_Snapshot_Month_Offset == 0')  # .tail(30)#.style.format(format_dict)#.style.format('{:,}'# .query('(End_of_Month == @current_eom)')

In [0]:
# Create new forecasting file with new forecast date
final_cols = ['Forecast_Date','End_of_Month','Relative_Month_Offset','Snapshot_Date','Relative_Snapshot_Month_Offset','Relative_Offset',
                                                        'Sub_Region_Code'] + cols
revenue_predict_new = final_merge_df1.copy()
#revenue_predict_new.insert(1, 'Forecast_Date', str(today.date()))
revenue_predict_new = convert_date_cols(revenue_predict_new)
#revenue_predict_new['Forecast_Date'] = pd.to_datetime(revenue_predict_new['Forecast_Date'])#.dt.date.astype(str)
#revenue_predict_new['Snapshot_Date'] = pd.to_datetime(revenue_predict_new['Snapshot_Date'])#.dt.date.astype(str)
#revenue_predict_new['End_of_Month'] = pd.to_datetime(revenue_predict_new['End_of_Month'])#.dt.date.astype(str)
revenue_predict_new = revenue_predict_new.sort_values(by=sort_cols_snp).reset_index(drop=True)
print('new:')
print(revenue_predict_new.info(verbose=True))
#reorder columns
#cols_order = [7,0,1,2,3,4,5,6]
#revenue_predict_new = revenue_predict_new[[revenue_predict_new.columns[i] for i in cols_order]]
filename = output_data_path + './revenue_region_forecast_' + str(pd.to_datetime(run_datetime).normalize().date()) + '.csv'
'''
if os.path.isfile(filename):
    revenue_predict_hist = pd.read_csv(filename, parse_dates=['Forecast_Date','Snapshot_Date_Short','End_of_Month'])
    print('history:')
    print(revenue_predict_hist.info(verbose=True))
    revenue_predict = pd.merge(revenue_predict_hist, revenue_predict_new, how='left', on=['Forecast_Date','Snapshot_Date_Short','End_of_Month'])
    print('revenue_predict:')
    print(revenue_predict.info(verbose=True))
    #revenue_predict['End_of_Month'] = pd.to_datetime(revenue_predict['End_of_Month'])
    #revenue_predict = revenue_predict.reset_index(drop=True)
else:
    revenue_predict = revenue_predict_new.copy()
'''
revenue_predict = revenue_predict_new.copy()    
revenue_predict = revenue_predict.sort_values(by=sort_cols_snp).reset_index(drop=True)
revenue_predict.to_csv(filename, header=True, index=False)
revenue_predict.tail(10)

In [0]:
from azureml.interpret import ExplanationClient

client = ExplanationClient.from_run(best_run)
engineered_explanations = client.download_model_explanation(raw=False)
#print(engineered_explanations.get_feature_importance_dict())
print("You can visualize the engineered explanations under the 'Explanations (preview)' tab in the AutoML run at:-\n" + best_run.get_portal_url())
raw_explanations = client.download_model_explanation(raw=True)
#print(raw_explanations.get_feature_importance_dict())
print("You can visualize the raw explanations under the 'Explanations (preview)' tab in the AutoML run at:-\n" + best_run.get_portal_url())

feature_imp_dict = pd.DataFrame(raw_explanations.get_feature_importance_dict().items())
feature_imp_dict.columns = ['Feature','Importance']
feature_imp_dict.plot()

In [0]:
# Customize Altair Theme to increase font size
import altair as alt

# alt.themes.enable('fivethirtyeight') #default, latimes, ggplot2, fivethirtyeight, urbaninstitute
theme = alt.themes.enable("latimes")
theme.fontsize = 14
theme
source = feature_imp_dict

title = "Forecast Model Drivers"

fontsize = 12
height = 600
width = 800

bars = (
    alt.Chart(source)
    .mark_bar()
    .encode(x="Importance:Q", y=alt.Y("Feature:O", sort="-x"))  # fontSize=fontsize
)

text = bars.mark_text(
    align="left",
    baseline="middle",  #    fontSize=fontsize,
    dx=3,  # Nudges text to right so it doesn't appear on top of the bar
).encode(text=alt.Text("Importance:Q", format=",.0f"))

(bars + text).properties(height=height, width=width, title=title)

In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df = revenue_predict.copy()
revenue_predict.query("Relative_Month_Offset >= 0").groupby("Relative_Offset").apply(
    lambda df: pd.Series(
        {
            "Mean Absolute Percent Error(MAPE)": MAPE(
                df[target_column_name], df["Predicted_Revenue"]
            ),
            "Root Mean Square Error (RMSE)": np.sqrt(
                mean_squared_error(df[target_column_name], df["Predicted_Revenue"])
            ),
            "Mean Absolute Error (MAE)": mean_absolute_error(
                df[target_column_name], df["Predicted_Revenue"]
            )
        }
    )
)


In [0]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

pd.set_option("display.float_format", lambda x: "%.2f" % x)

revenue_predict.query("Relative_Month_Offset < 0").groupby(
    "Snapshot_Date"
).apply(
    lambda df: pd.Series(
        {
            "Mean Absolute Percent Error(MAPE)": MAPE(
                df[target_column_name], df["Predicted_Revenue"]
            ),
            "Root Mean Square Error (RMSE)": np.sqrt(
                mean_squared_error(df[target_column_name], df["Predicted_Revenue"])
            ),
            "Mean Absolute Error (MAE)": mean_absolute_error(
                df[target_column_name], df["Predicted_Revenue"]
            ),
        }
    )
)


In [0]:
import altair as alt
#alt.themes.enable('fivethirtyeight') #default, latimes, ggplot2, fivethirtyeight, urbaninstitute
theme = alt.themes.enable('latimes')

title = "Actual vs Predicted Revenue"

source = final_merge_df.copy()
fontsize = 10
height = 450
width = 1000

x = 'yearmonth(End_of_Month):O'  # df['Importance'].astype(str)
y = 'sum(Predicted_Revenue):Q'  # df['Feature'].astype(str)
y1 = 'sum(Revenue):Q'
col = 'yearmonthdate(Snapshot_Date_Short):T' #.dt.date.astype(str) #'Snapshot_Date_Short'

ytransform ='datum.Predicted_Revenue / 1000000'
y1transform ='datum.Revenue / 1000000'


filter1 = "datum.Relative_Offset >= 0 && datum.Relative_Snapshot_Month_Offset ==0"
filter2 = "datum.Relative_Month_Offset < 0"
filter3 = "datum.Relative_Offset == 0 && datum.Relative_Month_Offset < 0"

base = alt.Chart(source).encode(
        alt.X(x, axis=alt.Axis(title="End_of_Month"))
)

bars = base.mark_bar().encode(  #color='#5276A7', #fontSize=fontsize
    alt.Y('sum(Predicted_Revenue_million):Q', axis=alt.Axis(title="Predicted_Revenue(Millions)")),
    #color=alt.Color('legend:O', scale=alt.Scale(range=['grey', 'blue'], domain=['Revenue_Million1', 'Predicted_Revenue_Million1'])),
    tooltip=[alt.Tooltip(y, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
bars_text = bars.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=15,  
    color='white'
).encode(
    text=alt.Text('sum(Predicted_Revenue_million):Q', format=',.1f')
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
#band = base.mark_errorband(extent="ci").encode(y=alt.Y("Predicted_Revenue:Q"))
bar = base.mark_bar(color='grey').encode(
    alt.Y('sum(Revenue_million):Q', axis=alt.Axis(title="Revenue(Millions)")),
    tooltip=[alt.Tooltip(y1, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)

#'''
bar_text = bar.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=15,  
    color='white'
).encode(
    text=alt.Text('sum(Revenue_million):Q', format=',.1f')
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)
#'''
alt_layers = alt.layer(bar + bar_text, bars + bars_text).properties(height=height, width=width, title=title) #resolve_scale(y='independent').
alt_layers

In [0]:
import altair as alt
#alt.themes.enable('fivethirtyeight') #default, latimes, ggplot2, fivethirtyeight, urbaninstitute
theme = alt.themes.enable('latimes')

title = "Actual vs Predicted Revenue with Confidence Interval"

source = final_merge_df.copy()
fontsize = 10
height = 450
width = 1000

x = 'yearmonth(End_of_Month):O'  # df['Importance'].astype(str)
y = 'sum(Predicted_Revenue):Q'  # df['Feature'].astype(str)
y1 = 'sum(Revenue):Q'
col = 'yearmonthdate(Snapshot_Date_Short):T' #.dt.date.astype(str) #'Snapshot_Date_Short'

ytransform ='datum.Predicted_Revenue / 1000000'
y1transform ='datum.Revenue / 1000000'


filter1 = "datum.Relative_Offset >= 0 && datum.Relative_Snapshot_Month_Offset == 0"
filter2 = "datum.Relative_Month_Offset < 0"
filter3 = "datum.Relative_Offset == 0 && datum.Relative_Month_Offset < 0"

base = alt.Chart(source).encode(
        alt.X(x, axis=alt.Axis(title="End_of_Month"))
)

lines = base.mark_line().encode(  #color='#5276A7', #fontSize=fontsize
    alt.Y('sum(Predicted_Revenue_million):Q', axis=alt.Axis(title="Predicted_Revenue(Millions)")),
    #color=alt.Color('legend:O', scale=alt.Scale(range=['grey', 'blue'], domain=['Revenue_Million1', 'Predicted_Revenue_Million1'])),
    tooltip=[alt.Tooltip(y, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
lines_text = lines.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=-15,  
    #color='white'
).encode(
    text=alt.Text('sum(Predicted_Revenue_million):Q', format=',.1f')
).transform_calculate(
  Predicted_Revenue_million=ytransform
).transform_filter(
  filter1
)
band = base.mark_errorband(extent="ci"
).encode(y=alt.Y('Predicted_Revenue_million:Q')
).transform_calculate(
  Predicted_Revenue_million=ytransform
)

line = base.mark_line(color='grey').encode(
    alt.Y('sum(Revenue_million):Q', axis=alt.Axis(title="Revenue(Millions)")),
    tooltip=[alt.Tooltip(y1, format=',.0f'),
             alt.Tooltip('End_of_Month', format='%b %Y')]
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)
#'''
line_text = line.mark_text(
    align='center',
    baseline='bottom',
    #fontSize=fontsize,
    #dx=18,
    dy=-15,  
    #color='white'
).encode(
    text=alt.Text('sum(Revenue_million):Q', format=',.1f')
).transform_calculate(
  Revenue_million=y1transform
).transform_filter(
  filter3
)
#'''
alt_layers = alt.layer(line + line_text, lines + lines_text, band).properties(height=height, width=width, title=title) #resolve_scale(y='independent').
alt_layers

In [0]:
title = "Actual vs Predicted Revenue"

source = revenue_predict.copy()
fontsize = 10
height = 375
width = 500

x = "yearmonth(End_of_Month):O"  # df['Importance'].astype(str)
y = "Predicted_Revenue:Q"  # df['Feature'].astype(str)
y1 = "Revenue:Q"
col = "yearmonth(Snapshot_Date):T"  # .dt.date.astype(str) #'Snapshot_Date_Short'

ytransform = "datum.Predicted_Revenue / 1000000"
y1transform = "datum.Revenue / 1000000"

base = alt.Chart(source).encode(x=x)

bars = (
    base.mark_bar()
    .encode(  # color='#5276A7',
        alt.Y("Predicted_Revenue_million:Q", axis=alt.Axis(titleColor="grey")),
        tooltip=[
            alt.Tooltip(y, format=",.0f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .transform_calculate(Predicted_Revenue_million=ytransform)
    .properties(width=width, height=height)
)



line = (
    base.mark_line(color="#F18727")
    .encode(
        alt.Y("Revenue_million:Q", axis=alt.Axis(titleColor="#F18727")),
        tooltip=[
            alt.Tooltip(y1, format=",.0f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .transform_calculate(Revenue_million=y1transform)
)

text = (
    bars.mark_text(
        align="center",
        baseline="bottom",
        # fontSize=fontsize,
        # dx=2,
        dy=15,
        color="white",
    )
    .encode(text=alt.Text("Predicted_Revenue_million:Q", format=",.1f"))
    .transform_calculate(Predicted_Revenue_million=ytransform)
)

alt.layer(bars + text, line, data=source).facet(facet=col, columns=2, title=title)

In [0]:
title = "Predicted Revenue Variance Percent with Error Bars"

source = (
    revenue_predict.copy()
)  # final_merge_df.query('Relative_Month_Offset >= -11').copy()
fontsize = 10
height = 600
width = 800

x = "yearmonth(End_of_Month):O"  # df['Importance'].astype(str)
y = "mean(Predicted_Revenue):Q"  # df['Feature'].astype(str)
y1 = "mean(Revenue):Q"
col = "yearmonth(Snapshot_Date):T"  # .dt.date.astype(str) #'Snapshot_Date_Short'

ytransform = "datum.Predicted_Revenue / 1000"
y1transform = "datum.Revenue / 1000"

base = alt.Chart(source).encode(x=x)

bars = (
    base.mark_bar()
    .encode(  # color='#5276A7',
        alt.Y(y),
        tooltip=[
            alt.Tooltip(y, format=",.2f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .properties(width=width, height=height)
)

error_bars = base.mark_errorbar(extent="ci").encode(y=alt.Y("Predicted_Revenue:Q"))

line = (
    base.mark_line(color="#F18727")
    .encode(
        alt.Y(y1, axis=alt.Axis(titleColor="#F18727")),
        tooltip=[
            alt.Tooltip(y1, format=",.2f"),
            alt.Tooltip("End_of_Month", format="%b %Y"),
        ],
    )
    .transform_calculate(Predicted_Revenue_Variance_1000=y1transform)
)

text = (
    bars.mark_text(
        align="center",
        baseline="bottom",
        # fontSize=fontsize,
        # dx=2,
        dy=15,
        color='white'
    )
    .encode(text=alt.Text("mean(Predicted_Revenue):Q", format=",.2f"))
    .transform_calculate(Predicted_Revenue_Variance_Percent=ytransform)
)

alt.layer(bars + error_bars + text, line, data=source).properties(height=height, width=width, title=title)

In [0]:
#**********************************************************STOP*************************************************************

In [0]:
'''
#Operationalization means getting the model into the cloud so that other can run it after you close the notebook. We will create a docker running on Azure Container Instances with the model.
description = 'AutoML Revenue forecaster'
tags = None
model = remote_run.register_model(model_name = model_name, description = description, tags = tags)

print(remote_run.model_id)

#Develop the scoring script
#For the deployment we need a function which will run the forecast on serialized data. It can be obtained from the best_run.
script_file_name = 'score_fcast.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file_name)

#Deploy the model as a Web Service on Azure Container 
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice
from azureml.core.model import Model

inference_config = InferenceConfig(environment = best_run.get_environment(), 
                                   entry_script = script_file_name)

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               tags = {'type': 'automl-forecasting'},
                                               description = 'Automl forecasting revenue service')

aci_service_name = 'automl-revenue-forecast-01'
print(aci_service_name)
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

aci_service.get_logs()

#Call the service
import json
X_query = X_test.copy()
# We have to convert datetime to string, because Timestamps cannot be serialized to JSON.
X_query[time_column_name] = X_query[time_column_name].astype(str)
# The Service object accept the complex dictionary, which is internally converted to JSON string.
# The section 'data' contains the data frame in the form of dictionary.
test_sample = json.dumps({'data': X_query.to_dict(orient='records')})
response = aci_service.run(input_data = test_sample)
# translate from networkese to datascientese
try: 
    res_dict = json.loads(response)
    y_fcst_all = pd.DataFrame(res_dict['index'])
    y_fcst_all[time_column_name] = pd.to_datetime(y_fcst_all[time_column_name], unit = 'ms')
    y_fcst_all['forecast'] = res_dict['forecast']    
except:
    print(res_dict)
    
y_fcst_all.head()    

#Delete the web service if desired
#serv = Webservice(ws, 'automl-revenue-forecast-01')
#serv.delete()     # don't do it accidentally
'''